In [1]:
import XGCN
from XGCN.data import io
from XGCN.utils.utils import ensure_dir, set_random_seed
import os.path as osp

## Prepare the raw data

Here we present a user-item graph example. The used gowalla dataset can be found 
in our XGCN repository: ``data/raw_gowalla/``, which is copied from LightGCN's official code repository: 
https://github.com/gusye1234/LightGCN-PyTorch.

We recommend to arrange the data with a clear directory structure. 
To get started, you may manually setup a ``XGCN_data`` directory as follows: 
(It's recommended to put your ``XGCN_data`` somewhere else than in this repository.)

```
XGCN_data
└── dataset
    └── raw_gowalla
        ├── train.txt
        └── test.txt
```